### **Obtener noticia**
_Por Juan Carlos Rodríguez-Raga y Andrés Mauricio Toloza Cruz_

Este código proporciona herramientas a través de Selenium y Newspaper3k para hacer web scrapping a noticias. 

##### **Primer paso: descargar e importar nuestras librerias**
Las librerias que estaremos usando son Selenium y Newspaper3k  para descargarlas usamos el siguiente código:

    ! pip install selenium
    ! pip install newspaper3k
    ! pip install lxml[html_clean]
    ! pip install pandas
    ! pip install openpyxl

Tambien usaremos os, datetime, time y urllib, pero esas ya vienen en nuestra instalación de Python

In [64]:
# Importamos las librerias necesarias

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium import webdriver
import time

import os

import newspaper
from newspaper import ArticleException

from urllib.parse import urlparse
import json

import pandas as pd

from datetime import datetime

##### **Segundo paso: extraer las noticias**
En este caso, vamos a extraer las noticias del último mes de la sección de "Política" de los siguientes medios:
1. El Espectador
2. Revista Semana
3. La Silla Vacía
4. Noticias Caracol
5. El País Colombia

Debido a que cada página tiene diferentes estrucutras, hacemos una función para cada sitio web

In [6]:
# Función scroll para cargar toda la web

def scroll():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

In [25]:
# Función para obtener las noticias

def get_news(url, parse_data):
	
    try: 

        article = newspaper.Article(url=url, language='es')
        article.download()
        article.parse()

        article_dict ={
            "title": str(article.title),
            "published_date": str(article.publish_date),
            "top_image": str(article.top_image),
            "link": urlparse(url).netloc,
            "full_link": str(url),
            "text": str(article.text)
        }

        parse_data[article_dict["title"]] = article_dict

        time.sleep(5)

    except ArticleException:
        article_dict ={
            "title": "Error",
            "published_date": "Error",
            "top_image": "Error",
            "link": urlparse(url).netloc,
            "full_link": str(url),
            "text": "Error"
            }

        parse_data[article_dict["title"]] = article_dict

            

        time.sleep(5)

In [42]:
# Función para ajustar el date 
def adjust_date(parse_data):
    for key in parse_data:
        if parse_data[key]["published_date"] == "Error":
            continue
        else:
            dt_str = parse_data[key]['published_date']
            try:
                dt = datetime.fromisoformat(dt_str)
                formatted_date = dt.strftime("%Y-%m-%d")
                parse_data[key]["published_date"] = formatted_date
            except ValueError:
                # Handle the case where the date format is incorrect
                parse_data[key]["published_date"] = "Error"

##### Extraer noticias de El Espectador

In [45]:
parse_data = {}
sentinela = True
num = 1
link = 'https://www.elespectador.com/archivo/politica/{pag}/'
    
while sentinela == True:
    
    resultados = []
    
    driver = webdriver.Chrome()
    driver.get(link.format(pag=num))
    time.sleep(3)
        
    scroll()
    url = driver.find_element(By.XPATH, '//*[@id="main-layout-12-13"]/div[1]/div/div[1]/div/div[2]/div[2]/a')
        
    resultados.append(url.get_attribute('href'))
    
    for i in range(1,6):
        url = driver.find_element(By.XPATH, '//*[@id="main-layout-12-13"]/div[3]/div/div['+str(i)+']/div/div[2]/h2/a')
        resultados.append(url.get_attribute('href'))
    
    for noticia in resultados:
        get_news(noticia, parse_data)  
    
    num += 1
    
    last_key = list(parse_data.keys())[-1]
    dt_str = parse_data[last_key]['published_date']
    
    try: 
        dt = datetime.fromisoformat(dt_str)
        print(dt)
        
        if dt.month != 1:
            sentinela = False
            
    except:
        print("Error verificando la fecha")
        continue
        
    driver.quit()
        

2025-01-26 01:53:51.834000+00:00
2025-01-25 20:51:20.773000+00:00
2025-01-24 21:49:18.778000+00:00
2025-01-24 14:13:00.741000+00:00
2025-01-24 01:14:44.642000+00:00
2025-01-23 18:32:12.129000+00:00
2025-01-23 11:38:38.448000+00:00
2025-01-22 22:41:44.960000+00:00
2025-01-22 15:15:06.911000+00:00
2025-01-22 12:05:00+00:00
2025-01-21 23:22:55.148000+00:00
2025-01-21 18:26:58.483000+00:00
2025-01-21 11:39:52.246000+00:00
2025-01-20 23:35:28.268000+00:00
2025-01-20 18:04:42.188000+00:00
2025-01-20 13:25:30.263000+00:00
2025-01-19 22:17:45.406000+00:00
2025-01-18 19:39:47.486000+00:00
2025-01-18 14:37:26.135000+00:00
2025-01-17 21:35:05.868000+00:00
2025-01-17 12:03:00+00:00
2025-01-16 22:24:31.081000+00:00
2025-01-16 14:47:55.109000+00:00
2025-01-16 01:57:34.765000+00:00
2025-01-15 20:05:16.570000+00:00
2025-01-15 12:59:03.378000+00:00
2025-01-14 23:24:35.194000+00:00
2025-01-14 16:58:25.789000+00:00
2025-01-14 00:16:26.546000+00:00
2025-01-13 15:05:00+00:00
2025-01-12 18:56:16.672000+00:0

In [58]:
# Ajustamos las fechas
adjust_date(parse_data)

# Eliminamos las noticias con fechas que no sean de enero

for key in list(parse_data.keys()):
    if parse_data[key]["published_date"] == "Error":
        continue
    else:
        dt_str = parse_data[key]['published_date']
        dt = datetime.fromisoformat(dt_str)
        if dt.month != 1:
            del parse_data[key]

In [59]:
# Convertimos el dicccionario en un DataFrame
df_extraidos = pd.DataFrame(list(parse_data.values()))

In [60]:
len(df_extraidos)

284

In [61]:
df_extraidos.head(5)

,title,published_date,top_image,link,full_link,text
0,“Me entristece ver a América Latina volviendo ...,2025-01-26,https://www.elespectador.com/pf/resources/imag...,www.elespectador.com,https://www.elespectador.com/colombia/me-entri...,"John Otis nació en 1962 en Minnesota, EE. UU.,..."
1,Petro a Trump: Colombia desautoriza entrada de...,2025-01-26,https://www.elespectador.com/resizer/v2/OOODBL...,www.elespectador.com,https://www.elespectador.com/politica/petro-tr...,“Los Estados Unidos no pueden tratar como deli...
2,“Ningún Estado se debilita por invertir en la ...,2025-01-26,https://www.elespectador.com/resizer/v2/LEELRO...,www.elespectador.com,https://www.elespectador.com/colombia-20/ningu...,"Vera Grabe advierte: “Existe, hoy, la tendenci..."
3,"Alto turmequé: sobre el fin del mundo, negocia...",2025-01-26,https://www.elespectador.com/resizer/v2/PY5UX2...,www.elespectador.com,https://www.elespectador.com/politica/alto-tur...,Mheo Foto: Mheo\n\nEl apocalipsis\n\nEn su cal...
4,Los cinco puntos que estableció el Gobierno pa...,2025-01-26,https://www.elespectador.com/resizer/v2/SZ3I4M...,www.elespectador.com,https://www.elespectador.com/politica/catatumb...,"El ministro del Interior, Juan Fernando Cristo..."


In [ ]:
# Guardamos el DataFrame en un archivo CSV y Excel

os.chdir('C:/Users/andre/OneDrive - Universidad de los andes/C.1. Centro de Datos/Noticias del mes/mes-en-noticias-1/')

df_extraidos.to_csv('raw-data/ElEspectador_Enero2025.csv', index=False)
df_extraidos.to_excel('raw-data/ElEspectador_Enero2025.xlsx', index=False)

##### Extraer noticias de Caracol

In [77]:
parse_data = {}
resultados = []

# Este código se encarga de unos Xpath específicos de la página de noticias de Caracol

driver = webdriver.Chrome()
driver.get('https://www.noticiascaracol.com/politica')
time.sleep(3)
    
url = driver.find_element(By.XPATH, '/html/body/bs-page/div[12]/div[2]/div/main/div/section[1]/ul/li[1]/div/ul/li[1]/ps-promo/div/div[2]/div[3]/h2/div/a')
resultados.append(url.get_attribute('href'))
    
for i in range(1,5):
    url = driver.find_element(By.XPATH, '/html/body/bs-page/div[12]/div[2]/div/main/div/section[1]/ul/li[1]/div/ul/li[2]/div['+str(i)+']/ps-promo/div/div[2]/div[3]/h2/div/a')
    resultados.append(url.get_attribute('href'))
    
driver.quit()

for noticia in resultados:
    get_news(noticia, parse_data) 


In [84]:
sentinela = True
num = 1

driver = webdriver.Chrome()
driver.get('https://www.noticiascaracol.com/politica')
time.sleep(3)

while sentinela == True:
    
    resultados = []

    url = driver.find_element(By.XPATH, '/html/body/bs-page/div[12]/div[2]/div/main/div/section[1]/ul/li[1]/ps-list-loadmore/ul/li['+str(num)+']/ps-promo/div/div[2]/div[3]/h2/div/a')
    resultados.append(url.get_attribute('href'))

    if num % 4 == 0:
        boton = driver.find_element(By.XPATH, '/html/body/bs-page/div[12]/div[2]/div/main/div/section[1]/ul/li[1]/ps-list-loadmore/div/a')
        driver.execute_script("arguments[0].scrollIntoView(true);", boton)
        driver.execute_script("arguments[0].click();", boton)
    
    for noticias in resultados:
        get_news(noticias, parse_data)
    
    num += 1
    
    last_key = list(parse_data.keys())[-1]
    dt_str = parse_data[last_key]['published_date']
    
    try: 
        dt = datetime.fromisoformat(dt_str)
        print(dt)
        
        if dt.month != 1:
            sentinela = False
            
    except:
        print("Error verificando la fecha")
        continue
        
driver.quit()


2025-01-26 16:06:28-05:00
2025-01-26 15:48:09-05:00
2025-01-26 14:26:54-05:00
2025-01-26 13:44:15-05:00
2025-01-26 13:26:23-05:00
2025-01-26 12:17:05-05:00
2025-01-26 11:21:32-05:00
2025-01-26 06:41:18-05:00
2025-01-24 09:45:34-05:00
2025-01-24 07:15:59-05:00
2025-01-23 18:18:32-05:00
2025-01-22 17:47:10-05:00
2025-01-22 14:58:05-05:00
2025-01-22 10:59:47-05:00
2025-01-22 06:48:20-05:00
2025-01-21 23:10:06-05:00
2025-01-21 20:35:36-05:00
2025-01-21 17:42:29-05:00
2025-01-21 11:03:25-05:00
2025-01-21 10:43:35-05:00
2025-01-21 09:43:48-05:00
2025-01-21 07:59:17-05:00
2025-01-20 19:33:02-05:00
2025-01-20 17:44:07-05:00
2025-01-20 14:54:24-05:00
2025-01-20 12:11:14-05:00
2025-01-20 10:59:26-05:00
2025-01-20 08:27:39-05:00
2025-01-18 21:07:27-05:00
2025-01-17 17:51:37-05:00
2025-01-17 09:43:08-05:00
2025-01-17 08:15:45-05:00
2025-01-16 17:12:16-05:00
2025-01-16 16:56:28-05:00
2025-01-16 16:29:39-05:00
2025-01-16 15:49:57-05:00
2025-01-16 11:59:05-05:00
2025-01-15 18:24:59-05:00
2025-01-15 1

In [85]:
len(parse_data)

73

In [87]:
# Ajustamos las fechas
adjust_date(parse_data)

# Eliminamos las noticias con fechas que no sean de enero

for key in list(parse_data.keys()):
    if parse_data[key]["published_date"] == "Error":
        continue
    else:
        dt_str = parse_data[key]['published_date']
        dt = datetime.fromisoformat(dt_str)
        if dt.month != 1:
            del parse_data[key]

In [88]:
# Convertimos el dicccionario en un DataFrame
df_extraidos = pd.DataFrame(list(parse_data.values()))

In [89]:
df_extraidos.head(5)

,title,published_date,top_image,link,full_link,text
0,"Laura Sarabia, canciller designada, hace “un l...",2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/laura...,"Laura Sarabia, canciller designada, hace “un l..."
1,Nicolás Maduro le ofrece al presidente Gustavo...,2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/nicol...,"NA través de Instagram, el autoproclamado mand..."
2,Federico Gutiérrez a Gustavo Petro tras sancio...,2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/feder...,Sigue la tensión internacional y nacional por ...
3,Alcalde Galán pide manejar crisis diplomática ...,2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/alcal...,"Este domingo, 26 de enero de 2025, el presiden..."
4,Petro le responde a Trump y ordena elevar aran...,2025-01-26,https://caracoltv.brightspotcdn.com/dims4/defa...,www.noticiascaracol.com,https://www.noticiascaracol.com/politica/petro...,"“Su bloqueo no me asusta”, fue la respuesta qu..."


In [90]:
# Guardamos el DataFrame en un archivo CSV y Excel

os.chdir('C:/Users/andre/OneDrive - Universidad de los andes/C.1. Centro de Datos/Noticias del mes/mes-en-noticias-1/')

df_extraidos.to_csv('raw-data/Caracol_Enero2025.csv', index=False)
df_extraidos.to_excel('raw-data/Caracol_Enero2025.xlsx', index=False)

##### Extraer noticias de La Silla Vacía

In [91]:
parse_data = {}
resultados = []

# Este código se encarga de unos Xpath específicos de la página de La Silla Vacía

driver = webdriver.Chrome()
driver.get('https://www.lasillavacia.com/silla-nacional/')
time.sleep(3)
      
for i in range(1,3):
    url = driver.find_element(By.XPATH, '//*[@id="post-24123"]/div/div[1]/div[1]/div/div/article['+str(i)+']/div/h2/a')
    resultados.append(url.get_attribute('href'))
    
driver.quit()

for noticia in resultados:
    get_news(noticia, parse_data) 


In [ ]:
# Este codigo es para extraer las noticias de las sección Nacion de La Silla Vacía

sentinela = True
num = 1

driver = webdriver.Chrome()
driver.get('https://www.lasillavacia.com/silla-nacional/')
time.sleep(3)

while sentinela == True:
    
    resultados = []

    url = driver.find_element(By.XPATH, '//*[@id="post-24123"]/div/div[2]/div/article['+str(num)+']/div/h3/a')
    resultados.append(url.get_attribute('href'))
    
    for noticias in resultados:
        get_news(noticias, parse_data)
    
    num += 1
    
    if num % 9 == 0:
        boton = driver.find_element(By.XPATH, '//*[@id="post-24123"]/div/div[2]/button/span[1]')
        driver.execute_script("arguments[0].scrollIntoView(true);", boton)
        driver.execute_script("arguments[0].click();", boton)
        time.sleep(5)
    
        last_key = list(parse_data.keys())[-1]
        dt_str = parse_data[last_key]['published_date']
        
        try: 
            dt = datetime.fromisoformat(dt_str)
            print(dt)
            
            if dt.month != 1:
                sentinela = False
                
        except:
            print("Error verificando la fecha")
            continue
        
driver.quit()

2025-01-20 17:50:10+00:00
2025-01-15 17:48:59+00:00
2025-01-09 18:02:01+00:00
2025-01-03 06:00:00+00:00
2024-12-23 06:00:00+00:00


In [99]:
# Este código es para sacar las noticias de la sección de En Vivo de La Silla Vacía

sentinela = True
link = 'https://www.lasillavacia.com/category/en-vivo/page/{num}/'
num = 1

while sentinela == True:
    
    resultados = []
    
    driver = webdriver.Chrome()
    driver.get(link.format(num=num))
    time.sleep(3)
    
    for i in range(1,11):
        url = driver.find_element(By.XPATH, '/html/body/div[1]/div/section/main/article['+str(i)+']/div/header/h2/a')
        resultados.append(url.get_attribute('href'))
    
    for noticia in resultados:
        get_news(noticia, parse_data)  
    
    num += 1
    
    last_key = list(parse_data.keys())[-1]
    dt_str = parse_data[last_key]['published_date']
    
    try: 
        dt = datetime.fromisoformat(dt_str)
        print(dt)
        
        if dt.month != 1:
            sentinela = False
            
    except:
        print("Error verificando la fecha")
        continue
        
    driver.quit()

2025-01-26 14:50:00+00:00
2025-01-24 20:51:48+00:00
2025-01-24 12:35:39+00:00
2025-01-23 20:16:36+00:00
2025-01-23 13:27:35+00:00
2025-01-22 18:18:36+00:00
2025-01-22 13:37:36+00:00
2025-01-21 19:43:54+00:00
2025-01-21 14:54:05+00:00
2025-01-20 21:15:35+00:00
2025-01-20 16:45:22+00:00
2025-01-19 19:42:32+00:00
2025-01-18 16:06:46+00:00
2025-01-17 18:24:26+00:00
2025-01-17 12:10:00+00:00
2025-01-16 19:37:15+00:00
2025-01-16 14:50:26+00:00
2025-01-15 21:54:15+00:00
2025-01-15 15:55:38+00:00
2025-01-14 23:11:26+00:00
2025-01-14 18:00:00+00:00
2025-01-13 20:15:53+00:00
2025-01-13 13:10:14+00:00
2025-01-11 14:39:02+00:00
2025-01-10 15:48:13+00:00
2025-01-10 15:48:13+00:00
2025-01-09 23:05:42+00:00
2025-01-09 17:28:13+00:00
2025-01-08 23:38:17+00:00
2025-01-08 16:41:49+00:00
2025-01-07 23:14:55+00:00
2025-01-07 18:00:50+00:00
2024-12-20 22:45:00+00:00


In [100]:
# Ajustamos las fechas
adjust_date(parse_data)

# Eliminamos las noticias con fechas que no sean de enero

for key in list(parse_data.keys()):
    if parse_data[key]["published_date"] == "Error":
        continue
    else:
        dt_str = parse_data[key]['published_date']
        dt = datetime.fromisoformat(dt_str)
        if dt.month != 1:
            del parse_data[key]

In [101]:
# Convertimos el dicccionario en un DataFrame
df_extraidos = pd.DataFrame(list(parse_data.values()))

In [102]:
df_extraidos.head(5)

,title,published_date,top_image,link,full_link,text
0,"Vicky madruga, pero también arriesga con su es...",2025-01-24,https://www.lasillavacia.com/wp-content/upload...,www.lasillavacia.com,https://www.lasillavacia.com/silla-nacional/vi...,“Los hinchas de Millonarios no podrán disfruta...
1,Cristo busca rearmar la coalición repartiendo ...,2025-01-24,https://www.lasillavacia.com/wp-content/upload...,www.lasillavacia.com,https://www.lasillavacia.com/silla-nacional/cr...,Armando Benedetti no va a ser el ministro de l...
2,Siete miembros clave del gabinete Trump para C...,2025-01-24,https://www.lasillavacia.com/wp-content/upload...,www.lasillavacia.com,https://www.lasillavacia.com/silla-nacional/si...,Marco Rubio Secretario de Estado\n\nEl senador...
3,Ofensiva del ELN en Catatumbo: la chispa de va...,2025-01-23,https://www.lasillavacia.com/wp-content/upload...,www.lasillavacia.com,https://www.lasillavacia.com/silla-nacional/la...,Hoy se cumple una semana desde que el ELN empe...
4,Cinco claves del pico histórico del turismo en...,2025-01-22,https://www.lasillavacia.com/wp-content/upload...,www.lasillavacia.com,https://www.lasillavacia.com/silla-nacional/ci...,El 2024 rompió récord de turismo. La cifra est...


In [103]:
len(df_extraidos)

354

In [104]:
# Guardamos el DataFrame en un archivo CSV y Excel

os.chdir('C:/Users/andre/OneDrive - Universidad de los andes/C.1. Centro de Datos/Noticias del mes/mes-en-noticias-1/')

df_extraidos.to_csv('raw-data/LaSillaVacia_Enero2025.csv', index=False)
df_extraidos.to_excel('raw-data/LaSillaVacia_Enero2025.xlsx', index=False)

##### Extraer noticias Semana 

In [105]:
# Este código es para extraer las noticias de la sección de Política de Semana

parse_data = {}

# Noticias con Xpath específicos

resultados = []
driver = webdriver.Chrome()
driver.get('https://www.semana.com/politica/')
time.sleep(3)

url = driver.find_element(By.XPATH, '//*[@id="fusion-app"]/div/main/div[1]/div[3]/div/div[1]/div[1]/div/div[1]/div/div[2]/div/h2/a')
resultados.append(url.get_attribute('href'))

for i in range(1,3):
    url = driver.find_element(By.XPATH, '//*[@id="fusion-app"]/div/main/div[1]/div[4]/div/div/div[1]/div/div['+str(i)+']/div[2]/h2/a')
    resultados.append(url.get_attribute('href'))
    
for i in range(1,3):
    url = driver.find_element(By.XPATH, '//*[@id="fusion-app"]/div/main/div[1]/div[4]/div/div/div[2]/div/div['+str(i)+']/div[2]/h2/a')
    resultados.append(url.get_attribute('href'))
    
for i in range(1,4):
    url = driver.find_element(By.XPATH, '//*[@id="fusion-app"]/div/main/div[1]/div[4]/div/div/div[3]/div/div['+str(i)+']/div[2]/h2/a')
    resultados.append(url.get_attribute('href'))
    
for noticia in resultados:
    get_news(noticia, parse_data)

driver.quit()

In [109]:
sentinela = True
num = 1

driver = webdriver.Chrome()
driver.get('https://www.semana.com/politica/')
time.sleep(3)

while sentinela == True:
    
    resultados = []
    
    url = driver.find_element(By.XPATH, '//*[@id="fusion-app"]/div/main/div[1]/div[6]/div[2]/div[1]/div/div[1]/div['+str(num)+']/div/div/div[2]/h2/a')
    resultados.append(url.get_attribute('href'))
    
    for noticias in resultados:
        get_news(noticias, parse_data)
        
    num += 1
    
    if num % 15 == 0:
        boton = driver.find_element(By.XPATH, '//*[@id="fusion-app"]/div/main/div[1]/div[6]/div[2]/div[1]/div/div[2]/a')
        driver.execute_script("arguments[0].scrollIntoView(true);", boton)
        driver.execute_script("arguments[0].click();", boton)
        time.sleep(5)
    
        last_key = list(parse_data.keys())[-1]
        dt_str = parse_data[last_key]['published_date']
        
        try: 
            dt = datetime.fromisoformat(dt_str)
            print(dt)
            
            if dt.month != 1:
                sentinela = False
                
        except:
            print("Error verificando la fecha")
            continue
        
driver.quit()

2025-01-25 17:11:12.255000+00:00
2025-01-24 18:48:16.280000+00:00
2025-01-24 00:50:12.702000+00:00
2025-01-23 11:40:29.485000+00:00
2025-01-22 16:10:58.995000+00:00
2025-01-21 23:41:45.580000+00:00
2025-01-21 17:35:44.835000+00:00
2025-01-21 00:51:52.429000+00:00
2025-01-20 17:03:45.980000+00:00
2025-01-19 10:49:18.982000+00:00
2025-01-18 04:13:09.246000+00:00
2025-01-17 16:58:00.158000+00:00
2025-01-16 20:57:07.623000+00:00
2025-01-16 11:15:56.631000+00:00
2025-01-15 16:55:43.911000+00:00
2025-01-15 00:13:26.062000+00:00
2025-01-14 17:14:01.129000+00:00
2025-01-13 14:09:59.156000+00:00
2025-01-11 17:38:22.499000+00:00
2025-01-11 14:01:36.858000+00:00
2025-01-10 14:51:35.650000+00:00
2025-01-09 14:23:26.465000+00:00
2025-01-08 14:52:44.401000+00:00
2025-01-08 15:06:00.353000+00:00
2025-01-04 14:28:39.787000+00:00
2024-12-31 17:26:07.473000+00:00


In [110]:
# Ajustamos las fechas
adjust_date(parse_data)

# Eliminamos las noticias con fechas que no sean de enero

for key in list(parse_data.keys()):
    if parse_data[key]["published_date"] == "Error":
        continue
    else:
        dt_str = parse_data[key]['published_date']
        dt = datetime.fromisoformat(dt_str)
        if dt.month != 1:
            del parse_data[key]

In [111]:
# Convertimos el dicccionario en un DataFrame
df_extraidos = pd.DataFrame(list(parse_data.values()))

In [112]:
df_extraidos.head(5)

,title,published_date,top_image,link,full_link,text
0,"“Nuestras exportaciones deben ampliarse”, Gust...",2025-01-26,https://www.semana.com/resizer/v2/WOQY2L2GWNDI...,www.semana.com,https://www.semana.com/politica/articulo/nuest...,El presidente Gustavo Petro anunció recienteme...
1,"Marco Rubio, secretario de Estado del gobierno...",2025-01-26,https://www.semana.com/resizer/v2/HK7EU2I45BD2...,www.semana.com,https://www.semana.com/politica/articulo/marco...,"El secretario de Estado de Estados Unidos, Mar..."
2,Petro citó a reunión urgente con el canciller ...,2025-01-26,https://www.semana.com/resizer/v2/VZOA7L6HOBEY...,www.semana.com,https://www.semana.com/politica/articulo/petro...,Altas fuentes de la Casa de Nariño le confirma...
3,Vicky Dávila: “Petro se puso a jugar con cande...,2025-01-26,https://www.semana.com/resizer/v2/VXI55AVUANEI...,www.semana.com,https://www.semana.com/politica/articulo/vicky...,"En una declaración contundente, Vicky Dávila a..."
4,Petro ordenó el uso del avión presidencial par...,2025-01-26,https://www.semana.com/resizer/v2/T3ZUF4S2JVFS...,www.semana.com,https://www.semana.com/politica/articulo/petro...,Fuentes de alto nivel de la Casa de Nariño die...


In [113]:
len(df_extraidos)

395

In [114]:
# Guardamos el DataFrame en un archivo CSV y Excel

os.chdir('C:/Users/andre/OneDrive - Universidad de los andes/C.1. Centro de Datos/Noticias del mes/mes-en-noticias-1/')

df_extraidos.to_csv('raw-data/Semana_Enero2025.csv', index=False)
df_extraidos.to_excel('raw-data/Semana_Enero2025.xlsx', index=False)